In [1]:
# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
if 'GPU' in str(device_lib.list_local_devices()): print ('Remote Machine') 
else: print ('Local Machine')

Local Machine


In [2]:
# Pre-process

'''
.txt to feature and target vectors
Handling mixed feature data: Categorical and Continous
One-hot encoding target data
'''

# Model

'''
Overview: 
    8 dim feature vector: [6 continous, 2 categorical]
        [Pos.x, Pos.y, Vel.x, Vel.y, Lander.angle, Lander.angularVel, ifLeftLegDown, ifRightLegDown]
        https://github.com/openai/gym/blob/5e6f11d07781404459672e114cb86cb6a1a015c6/gym/envs/box2d/lunar_lander.py#L285
    ??? Model architecture:

    8 dim target vector: 
        [000, 100, 010, 001, 110, 101, 011, 111]
        [No Action, Left, Up, Right, Left-Up, Left-Right, Right-Up, All Actions]
'''

'\nOverview: \n    8 dim feature vector: [6 continous, 2 categorical]\n        [Pos.x, Pos.y, Vel.x, Vel.y, Lander.angle, Lander.angularVel, ifLeftLegDown, ifRightLegDown]\n        https://github.com/openai/gym/blob/5e6f11d07781404459672e114cb86cb6a1a015c6/gym/envs/box2d/lunar_lander.py#L285\n    ??? Model architecture:\n\n    8 dim target vector: \n        [000, 100, 010, 001, 110, 101, 011, 111]\n        [No Action, Left, Up, Right, Left-Up, Left-Right, Right-Up, All Actions]\n'

In [3]:
import json
import numpy as np

In [2]:
with open(r'../../../data/lunarlander/sample_lunarlander.tsv',"r") as f:
    raw_dataset = f.readlines()

In [1]:
raw_dataset

NameError: name 'raw_dataset' is not defined

In [19]:
def split_data(data,train_fraction=0.7):
    data_len = len(data)
    split_index = int(data_len*train_fraction)
    return data[:split_index],data[split_index:]

In [20]:
x_full = []
y_full = []
score_full = []

for line in raw_dataset:
    if line == "---\n":
        continue
    temp_line = json.loads(line)
    x_full.append(temp_line[0])
    y_full.append(temp_line[1])
    score_full.append(temp_line[2])
    
    
x_train,x_test = split_data(x_full,0.8)
y_train,y_test = split_data(y_full,0.8)
    
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [21]:
print("Training Data Size : ",x_train.shape)
print("Testing Data Size : ",x_test.shape)

Training Data Size :  (27263, 8)
Testing Data Size :  (6816, 8)


In [8]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=0)

In [9]:
clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=2, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [10]:
print(clf.feature_importances_)

[0.00725455 0.26810567 0.04007069 0.1445603  0.13582232 0.04685554
 0.17688887 0.18044205]


In [13]:
import pickle
filename = 'forest_model.sav'
pickle.dump(clf, open(filename, 'wb'))

In [25]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

bdt_real = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=7),
    n_estimators=600,
    learning_rate=1)

bdt_discrete = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2),
    n_estimators=600,
    learning_rate=1.5,
    algorithm="SAMME")

In [26]:
bdt_real.fit(x_train, y_train)

ValueError: bad input shape (27263, 3)

TypeError: 'tuple' object is not callable